# TD4 : classification à l'aide d'un RNN

Ce notebook a été développé dans le cours donné par J. Velcin et J. Cugliari sur le Deep Learning à l'Université de Lyon 2.

Les données en entrée sont toujours les mêmes que pour les TD 2 et 3.

In [3]:
import os
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

with open(os.path.join("SMSSpamCollection.txt")) as f:
    lines = [line.strip().split("\t") for line in f.readlines()]

text = [x[1] for x in lines]
y = [int(x[0] == "spam") for x in lines]

vocab_size = 10000
max_length = 10

encoded_docs = [one_hot(d, vocab_size) for d in text]
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [4]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(padded_docs, y, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=123)

Cette fois, on remplace le MLP par un réseau "simple" récurrent comportant 4 cellules.

In [5]:
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model_rnn = Sequential()
model_rnn.add(SimpleRNN(4, return_sequences=False, input_shape=(max_length, vocab_size)))
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# l'optimiseur "rmsprop" est considéré comme plus efficace pour les RNN

print(model_rnn.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4)                 40020     
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 40,025
Trainable params: 40,025
Non-trainable params: 0
_________________________________________________________________
None


Quelques observations sur ce réseau :

- L'entrée est donnée sous la forme d'un tenseur (nombre d'observations, nombre de mots dans une séquence, nombre de caractéristiques pour un mot).

- Pour chaque observation (texte de n mots), une seule sortie est attendue (return_sequences=False).

- Ici, les deux dernières couches comportent respectivement 4 cellules (+1 pour le biais) et 1 cellule pour la sortie finale (classification binaire).

On a besoin de transformer l'index de chaque mot dans sa représentation binaire correspondante (on parle de "one hot encoding").

In [ ]:
from tensorflow.keras.utils import to_categorical

train_X_binary = to_categorical(train_X, num_classes=vocab_size)
test_X_binary = to_categorical(test_X, num_classes=vocab_size)

In [ ]:
train_X_binary.shape

In [ ]:
test_X_binary.shape

In [ ]:
# il peut être parfois nécessaire de "ré-arranger" la forme du tenseur...
#train_X_binary = train_X_binary.reshape((train_X.shape[0], 10, 10000))
#test_X_binary = test_X_binary.reshape((test_X.shape[0], 10, 10000))

import numpy as np
train_y = np.array(train_y)
test_y = np.array(test_y)

On peut à présent entraîner le modèle, en faisant attention à bien prendre la version "binarisée" des données d'entrée.

In [ ]:
model_rnn.fit(train_X_binary, train_y, batch_size=10, epochs=10, verbose=1)

On peut regarder la prédiction du modèle sur les données de test.

In [ ]:
loss, accuracy = model_rnn.evaluate(test_X_binary, test_y, verbose=1)
print()
print('Accuracy: %f' % (accuracy*100))

On peut bien sûr s'arrêter à l'étape de prédiction afin de mieux contrôler l'utilisation / l'évaluation du modèle.

In [ ]:
pred_y_rnn = model_rnn.predict(test_X_binary, verbose=1)

In [ ]:
print(len(pred_y_rnn))
print(pred_y_rnn[0:10])
print(test_y[0:10])

In [ ]:
from sklearn.metrics import zero_one_loss

pred_y_rnn_binary = [int(x) for x in (pred_y_rnn > 0.5)]
e=zero_one_loss(test_y, pred_y_rnn_binary)
print(e)

On peut bien sûr ajouter une couche de plongement de mots (word embedding) au RNN.

Il s'agit de ne pas oublier de revenir à une représentation de type "index" pour l'entrée.

In [ ]:
from tensorflow.keras.layers import Embedding

size_embedding = 20

model_rnn_with_embedding = Sequential()
# here is the additional layer (note that we add 50*10000 parameters!)
model_rnn_with_embedding.add(Embedding(output_dim=size_embedding, input_dim=vocab_size, input_length=max_length))
model_rnn_with_embedding.add(SimpleRNN(4, return_sequences=False))
model_rnn_with_embedding.add(Dense(1, activation='sigmoid'))
model_rnn_with_embedding.compile(optimizer='rmsprop', loss='binary_crossentropy')

print(model_rnn_with_embedding.summary())

model_rnn_with_embedding.fit(train_X, train_y, batch_size=10, epochs=10, verbose=1)

In [ ]:
pred_y_rnn_with_embedding = model_rnn_with_embedding.predict(test_X, verbose=1)
pred_y_rnn_binary_with_embedding = [int(x) for x in (pred_y_rnn_with_embedding > 0.5)]
e=zero_one_loss(test_y, pred_y_rnn_binary_with_embedding)
print()
print(e)

A noter qu'il est toujours possible d'initialiser (bootstrap) la couche d'embedding à l'aide des représentations estimées sur un corpus externe (ex. wikipedia).